In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import backend as K
import segmentation_models as sm
import UNet
import UNet3D
import EfficientUNet

Segmentation Models: using `tf.keras` framework.


In [2]:
tf.__version__

'2.3.0'

### Short model information

In [3]:
def model_info(model):
    print(model.input_shape)
    print(model.output_shape)
    params = model.count_params()
    print(f"{params:,}")
    
def model_memory_usage(batch_size, model):
    shapes_mem_count = 0
    internal_model_mem_count = 0
    for l in model.layers:
        layer_type = l.__class__.__name__
        if layer_type == 'Model':
            internal_model_mem_count += get_model_memory_usage(batch_size, l)
        single_layer_mem = 1
        out_shape = l.output_shape
        if type(out_shape) is list:
            out_shape = out_shape[0]
        for s in out_shape:
            if s is None:
                continue
            single_layer_mem *= s
        shapes_mem_count += single_layer_mem

    trainable_count = np.sum([K.count_params(p) for p in model.trainable_weights])
    non_trainable_count = np.sum([K.count_params(p) for p in model.non_trainable_weights])

    number_size = 4.0
    if K.floatx() == 'float16':
        number_size = 2.0
    if K.floatx() == 'float64':
        number_size = 8.0

    total_memory = number_size * (batch_size * shapes_mem_count + trainable_count + non_trainable_count)
    gbytes = np.round(total_memory / (1024.0 ** 3), 3) + internal_model_mem_count
    print(gbytes)

### Used models

In [4]:
UNET2D_1 = UNet.UNet(input_shape=(256, 256, 1), output_classes=9, filters=64, depth=5, conv_per_block=2, padding='same', dropouts=0.50, batch_normalization=True)

In [5]:
UNET3D_1 = UNet3D.UNet(input_shape=(256, 256, 8, 1), output_classes=9, output_activation='sigmoid', filters=32, depth=4, pool_size=(2, 2, 2), conv_per_block=2, dropouts=0.50, batch_normalization=True)

In [6]:
UNET3D_2 = UNet3D.UNet(input_shape=(256, 256, 16, 1), output_classes=9, output_activation='sigmoid', filters=32, depth=3, pool_size=(2, 2, 2), conv_per_block=2, dropouts=0.50, batch_normalization=True)

In [7]:
model_info(UNET2D_1)
model_memory_usage(1, UNET2D_1)
model_info(UNET3D_1)
model_memory_usage(1, UNET3D_1)
model_info(UNET3D_2)
model_memory_usage(1, UNET3D_2)

(None, 256, 256, 1)
(None, 256, 256, 9)
31,054,665
0.453
(None, 256, 256, 8, 1)
(None, 256, 256, 8, 9)
5,841,929
1.123
(None, 256, 256, 16, 1)
(None, 256, 256, 16, 9)
1,414,409
2.134


### UNet

In [8]:
UN_1 = UNet.UNet(input_shape=(256, 256, 3), output_classes=4, filters=64, depth=5, conv_per_block=2, padding='same', dropouts=0.50, batch_normalization=True)
UN_2 = UNet.UNet(input_shape=(572, 572, 3), output_classes=4, filters=64, depth=5, conv_per_block=2, padding='valid', dropouts=[0.10, 0.10, 0.10, 0.20, 0.50, 0.20, 0.10, 0.10, 0.10], batch_normalization=True)
UN_3 = UNet.UNet(input_shape=(256, 256, 3), output_classes=4, filters=32, depth=5, conv_per_block=2, padding='same', dropouts=0.50, batch_normalization=False)
UN_4 = UNet.UNet(input_shape=(256, 256, 2), output_classes=4, filters=64, depth=5, conv_per_block=2, padding='same', dropouts=0.50, batch_normalization=True, groups=1)
UN_5 = UNet.UNet(input_shape=(256, 256, 2), output_classes=4, filters=64, depth=5, conv_per_block=2, padding='same', dropouts=0.50, batch_normalization=True, groups=2)

In [9]:
model_info(UN_1)

(None, 256, 256, 3)
(None, 256, 256, 4)
31,055,492


In [10]:
model_info(UN_2)

(None, 572, 572, 3)
(None, 388, 388, 4)
31,055,492


In [11]:
model_info(UN_3)

(None, 256, 256, 3)
(None, 256, 256, 4)
7,760,196


In [12]:
model_info(UN_4)
model_info(UN_5)

(None, 256, 256, 2)
(None, 256, 256, 4)
31,054,916
(None, 256, 256, 2)
(None, 256, 256, 4)
31,035,908


### UNet3D

In [13]:
UN3D_1 = UNet3D.UNet(input_shape=(64, 64, 64, 1), output_classes=9, output_activation='sigmoid', filters=32, depth=4, pool_size=(2, 2, 2), conv_per_block=2, dropouts=0.50, batch_normalization=True)
UN3D_2 = UNet3D.UNet(input_shape=(256, 256, 32, 1), output_classes=9, output_activation='sigmoid', filters=32, depth=4, pool_size=(2, 2, 2), conv_per_block=2, dropouts=0.50, batch_normalization=True)
UN3D_3 = UNet3D.UNet(input_shape=(256, 256, 4, 1), output_classes=9, output_activation='sigmoid', filters=32, depth=4, pool_size=[(2, 2, 2), (2, 2, 1), (2, 2, 1), None, (2, 2, 1), (2, 2, 1), (2, 2, 2)], conv_per_block=2, dropouts=0.50, batch_normalization=True)
UN3D_4 = UNet3D.UNet(input_shape=(256, 256, 5, 1), output_classes=9, output_activation='sigmoid', filters=64, depth=4, pool_size=(2, 2, 1), conv_per_block=2, dropouts=[0.10, 0.10, 0.20, 0.50, 0.20, 0.10, 0.10], batch_normalization=True)
UN3D_5 = UNet3D.UNet(input_shape=(256, 256, 16, 1), output_classes=9, output_activation='sigmoid', filters=32, depth=4, pool_size=(2, 2, 2), conv_per_block=2, dropouts=0.50, batch_normalization=True)

In [14]:
model_info(UN3D_1)
model_info(UN3D_2)
model_info(UN3D_3)
model_info(UN3D_4)
model_info(UN3D_5)

(None, 64, 64, 64, 1)
(None, 64, 64, 64, 9)
5,841,929
(None, 256, 256, 32, 1)
(None, 256, 256, 32, 9)
5,841,929
(None, 256, 256, 4, 1)
(None, 256, 256, 4, 9)
5,841,929
(None, 256, 256, 5, 1)
(None, 256, 256, 5, 9)
23,351,305
(None, 256, 256, 16, 1)
(None, 256, 256, 16, 9)
5,841,929


### EfficientUNet

In [15]:
EfficientNetB0 = tf.keras.applications.EfficientNetB0(
    include_top=None,
    weights=None,
    input_tensor=None,
    input_shape=(256, 256, 3),
    pooling=None,
    classes=None,
    classifier_activation=None
    )

EUNB0_1 = EfficientUNet.EfficientUNetB0((256, 256, 1), 2, weight_name=None)
EUNB0_2 = EfficientUNet.EfficientUNetB0((256, 256, 3), 6, weight_name=None)
EUNB0_3 = EfficientUNet.EfficientUNetB0((256, 256, 10), 32, weight_name=None)

EUNB0 = EfficientUNet.EfficientUNetB0((256, 256, 3), 4, weight_name=None)
EUNB1 = EfficientUNet.EfficientUNetB1((256, 256, 3), 4, weight_name=None)
EUNB2 = EfficientUNet.EfficientUNetB2((256, 256, 3), 4, weight_name=None)
EUNB3 = EfficientUNet.EfficientUNetB3((256, 256, 3), 4, weight_name=None)
EUNB4 = EfficientUNet.EfficientUNetB4((256, 256, 3), 4, weight_name=None)
EUNB5 = EfficientUNet.EfficientUNetB5((256, 256, 3), 4, weight_name=None)
EUNB6 = EfficientUNet.EfficientUNetB6((256, 256, 3), 4, weight_name=None)
EUNB7 = EfficientUNet.EfficientUNetB7((256, 256, 3), 4, weight_name=None)

In [16]:
model_info(EfficientNetB0)

(None, 256, 256, 3)
(None, 8, 8, 1280)
4,049,571


In [17]:
model_info(EUNB0_1)

(None, 256, 256, 1)
(None, 256, 256, 2)
15,390,283


In [18]:
model_info(EUNB0_2)

(None, 256, 256, 3)
(None, 256, 256, 6)
15,390,409


In [19]:
model_info(EUNB0_3)

(None, 256, 256, 10)
(None, 256, 256, 32)
15,391,300


In [20]:
model_info(EUNB0)

(None, 256, 256, 3)
(None, 256, 256, 4)
15,390,343


In [21]:
model_info(EUNB1)

(None, 256, 256, 3)
(None, 256, 256, 4)
17,916,011


In [22]:
model_info(EUNB2)

(None, 256, 256, 3)
(None, 256, 256, 4)
20,030,941


In [23]:
model_info(EUNB3)

(None, 256, 256, 3)
(None, 256, 256, 4)
24,161,043


In [24]:
model_info(EUNB4)

(None, 256, 256, 3)
(None, 256, 256, 4)
33,005,123


In [25]:
model_info(EUNB5)

(None, 256, 256, 3)
(None, 256, 256, 4)
45,632,731


In [26]:
model_info(EUNB6)

(None, 256, 256, 3)
(None, 256, 256, 4)
60,060,787


In [27]:
model_info(EUNB7)

(None, 256, 256, 3)
(None, 256, 256, 4)
85,207,419
